## Sentiment Analysis

In [1]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [2]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

print(get_available_devices())

2022-10-29 02:16:29.201713: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 02:16:29.367333: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-29 02:16:30.034607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib.real:/opt/conda/lib/python3.8/site-packages/torch/lib:/opt/conda/lib/python3.8/site-packages/torch_tensorrt/lib:/usr/local/cuda/compat/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64


['/device:CPU:0', '/device:GPU:0', '/device:GPU:1', '/device:GPU:2', '/device:GPU:3', '/device:GPU:4', '/device:GPU:5', '/device:GPU:6', '/device:GPU:7']


2022-10-29 02:16:30.963592: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-29 02:16:35.695407: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 7065 MB memory:  -> device: 0, name: A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0
2022-10-29 02:16:35.697638: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:1 with 16449 MB memory:  -> device: 1, name: A100-SXM4-40GB, pci bus id: 0000:0f:00.0, compute capability: 8.0
2022-10-29 02:16:35.699515: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:2 with 30692 MB memory:  -> device: 2, name: A100-SXM4-40GB, pci bus id: 0000:47:00.0, compute capabi

In [3]:
# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [5]:
import pandas as pd
import numpy as np
import torch
import codecs
import time
from nltk.tokenize import sent_tokenize
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from collections import Counter

/opt/conda/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
%cd ..

/workspace/asurion_f22_jw/Asurion-customer-propensity


In [7]:
df = pd.read_csv('data/transcript_may_encoded_100.csv')
# df = pd.read_csv('data/transcript_march_collated_encoded.csv')
df = df.drop(columns='Unnamed: 0')
# df = df[df['accepted_flg'] == True]
df['number_of_words'] = df.encrypted_collated_transcription.apply(lambda x: len(x.split()))
df.head()

,ID,sales_offer_date,accepted_flg,encrypted_collated_transcription,number_of_words
0,4f81e2619aae688a3a814da7f58afdecb9720e9fdd5070...,2022-05-31,False,Thank you for calling [CLIENT] Tech for speaki...,113
1,03bcb2c2dd3e29b8b7ba6c0cf8c7232c8637d6bc73a760...,2022-05-13,False,[CLIENT] tech My name is [NAME]. May I have yo...,454
2,c1eeaa4c1fe8030bb6f0deaa81a13b468001c699586f46...,2022-05-29,False,Hi. Thank you for call. tech you're speaking w...,1674
3,5bf5d93c818534d9edcff8cbb45f28d3e0438cf7ab7850...,2022-05-22,False,Thank you for calling [CLIENT] Tech Coach. My ...,1129
4,c484a81de8c25bcfb9d95f36976cfb425a3d2c7602a93c...,2022-05-21,False,Hi. Thank you for calling [CLIENT] Barcode Coa...,668


In [8]:
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package basque_grammars is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Pac

True

In [9]:
def sentence_token_nltk(str):
    sent_tokenize_list = sent_tokenize(str)
    return sent_tokenize_list

In [10]:
# pip install stanza

In [11]:
# import stanza
# def sentence_token_stanza(str):
#     nlp = stanza.Pipeline(lang='en', processors='tokenize')
#     doc = nlp(str)
#     sent_tokenize_list = [sentence.text for sentence in doc.sentences]
#     return sent_tokenize_list

In [12]:
# Import generic wrappers
# from transformers import AutoModel, AutoTokenizer 

from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Define the model repo
myckpt = 'cardiffnlp/twitter-roberta-base-sentiment'
mytokenizer = AutoTokenizer
device = torch.device(str("cuda:1") if torch.cuda.is_available() else "cpu") 

# Download pytorch model
model = AutoModelForSequenceClassification.from_pretrained(myckpt).to(device)

In [13]:
def tokenize_inputs(text, checkpoint = myckpt, t=mytokenizer):
    tokenizer = t.from_pretrained(myckpt, is_split_into_words = True, model_max_length=512)
    return tokenizer(text, truncation=True)

In [17]:
def sa_by_sent(df, text_col:str):
    '''
    Function to do sentiment analysis by sentences of each transcript
    '''
    start=time.time()
    df_texts_list=df[text_col].to_list()
    sent_lists=[sentence_token_nltk(sent) for sent in df_texts_list]
    tok_sents=[tokenize_inputs(sent_list) for sent_list in sent_lists]

    for idx in range(len(tok_sents)):
        winner_list=[]
        
        for i in range(len(tok_sents[idx]['input_ids'])):
            input_list=[tok_sents[idx]['input_ids'][i]]
            attention_list=[tok_sents[idx]['attention_mask'][i]]
            input_ids = torch.FloatTensor(input_list).to(device)
            attention_mask = torch.FloatTensor(attention_list).to(device)
            input_dict = {'input_ids': input_ids.long(),'attention_mask': attention_mask.int()}
            preds = model(**input_dict)
            probs = torch.nn.functional.softmax(preds[0], dim=-1)
            winner = torch.argmax(probs).item()
            winner_list.append(winner)
            
    #         Labels: 0 -> Negative; 1 -> Neutral; 2 -> Positive
        df.loc[idx, 'sentiment'] = ['Negative', 'Neutral', 'Positive'][Counter(winner_list).most_common(1)[0][0]]
    print(round(time.time()-start, 2), 'seconds')
    return df

In [18]:
df1=sa_by_sent(df = df, text_col='encrypted_collated_transcription')

100.47 seconds


In [19]:
df1.to_csv('sa_w_roberta_by_sent.csv', index=0)

In [20]:
df1

,ID,sales_offer_date,accepted_flg,encrypted_collated_transcription,number_of_words,sentiment
0,4f81e2619aae688a3a814da7f58afdecb9720e9fdd5070...,2022-05-31,False,Thank you for calling [CLIENT] Tech for speaki...,113,Neutral
1,03bcb2c2dd3e29b8b7ba6c0cf8c7232c8637d6bc73a760...,2022-05-13,False,[CLIENT] tech My name is [NAME]. May I have yo...,454,Neutral
2,c1eeaa4c1fe8030bb6f0deaa81a13b468001c699586f46...,2022-05-29,False,Hi. Thank you for call. tech you're speaking w...,1674,Neutral
3,5bf5d93c818534d9edcff8cbb45f28d3e0438cf7ab7850...,2022-05-22,False,Thank you for calling [CLIENT] Tech Coach. My ...,1129,Neutral
4,c484a81de8c25bcfb9d95f36976cfb425a3d2c7602a93c...,2022-05-21,False,Hi. Thank you for calling [CLIENT] Barcode Coa...,668,Neutral
5,b6716b08ca0380e7b5c7e9b78fcecf5787ccf3d61d1869...,2022-05-03,False,Thank you for calling Coach. This is Sean spea...,2967,Neutral
6,9f900ab27f1f249ce94d45947421b50ce9b9a85ec0d909...,2022-05-17,True,Thank you for calling [CLIENT] Tech Coach. Ben...,7032,Neutral
7,143fada0c669611e23e414d405a4f528a2528718a20c00...,2022-05-03,False,Morning. Thanks for [CLIENT] Tech Coach. My na...,2561,Neutral
8,8cbccda49661f95352e153157f8894c4d2d8e52d28740b...,2022-05-09,False,thank Hello? Hi. Thank you for calling [CLIENT...,1661,Neutral
9,46f3a9e86aa971a85d0d4cda2a886233d88f9e1da57eff...,2022-05-19,False,Hi. Thank you for [CLIENT] Tech Coach. My name...,536,Neutral


In [21]:
# df.loc[26, 'encrypted_collated_transcription']

In [22]:
max(df['number_of_words'])

7032